In [1]:
from pathlib import Path

from rasterstats import zonal_stats
import rasterio as rio
import fiona
import geopandas as gpd
import pandas as pd

import matplotlib.pyplot as plt

from metric_comp.additional_stats import additional_statistics_functions

In [22]:
def define_path(template: str, year: int, band: int, viirs_metric_names) -> Path:
    """Construct the file path for the given template, year, and band."""
    metric_name = viirs_metric_names[band - 1]
    print(template, band, year, metric_name)
    return Path(template.format(band=band, year=year, metric_name=metric_name))

def load_tiff(template: str, year: int, band: int, viirs_metric_names):
    tiff_path = define_path(template, year, band, viirs_metric_names)
    try:
        with rio.open(tiff_path) as src:
            src_transform = src.transform
            src_array = src.read(1)
    except:
        print(f"Cannot find {tiff_path}, probably no metric tiff file...")
        return None
    print("Dataset shape:", src.shape)
    print("Dataset metadata:", src.meta)
    print(f"Dataset metric transform: \n{src_transform}")
    return src_array, src_transform

def get_zonal_stats_df(ds1_template: str, ds2_template: str, year: int, band: int, shapefile, shp_id_column, viirs_metric_names, additional_statistics_functions=additional_statistics_functions):
    ds1_array, ds1_transform = load_tiff(ds1_template, year, band, viirs_metric_names)
    ds2_array, ds2_transform = load_tiff(ds2_template, year, band, viirs_metric_names)
    if ds1_array is None or ds2_array is None:
        print(f"At least one of the two datasets did not open.")
        return None
    ds1_stats = zonal_stats(shapefile, ds1_array, affine=ds1_transform, add_stats=additional_statistics_functions)
    ds2_stats = zonal_stats(shapefile, ds2_array, affine=ds2_transform, add_stats=additional_statistics_functions)
    ds1_stats_df = pd.DataFrame(ds1_stats)
    ds1_stats_df[shp_id_column] = shapefile[shp_id_column]
    ds2_stats_df = pd.DataFrame(ds2_stats)
    ds2_stats_df[shp_id_column] = shapefile[shp_id_column]  
    print('assigning sensor and version manually')
    ds1_stats_df['sensor'] = 'modis'
    ds1_stats_df['version'] = 'new-v6'
    ds2_stats_df['sensor'] = 'viirs'
    ds2_stats_df['version'] = globals().get('viirs_metric_version', 'v1')
    combined_stats = pd.concat([ds1_stats_df, ds2_stats_df])
    return combined_stats

In [4]:

import geopandas as gpd
import os
# Using geopandas to open the shapefile
interior_refuges_shp_path = '/Users/ojlarson/Desktop/yf/Interior_Refuges.shp'
interior_refuges = gpd.read_file(interior_refuges_shp_path)
#interior_refuges.set_geometry('geometry', crs=interior_refuges.crs, inplace=True)
print(interior_refuges)
shp_name = os.path.splitext(os.path.basename(interior_refuges_shp_path))[0]
shp_column = 

   FWSReg  IFWSNo      NWRName NWRCode UnitName UnitCode SubunitNam  \
0       7     995       Innoko     inn   Kaiyuh      kai       None   
1       7     997       Kanuti     kan     None     None       None   
2       7    1000      Koyukuk     kuk     None     None       None   
3       7    1001      Nowitna     now     None     None       None   
4       7    1003       Tetlin     tet     None     None       None   
5       7    1006  Yukon Flats     ykf     None     None       None   

      IslandName                      Source    Scale      AcresGIS  \
0           None                  USGS quads  1:63360  9.855636e+01   
1           None  USGS paper quads, BLM PLSS  1:63360  1.636828e+06   
2           None  USGS paper quads, BLM PLSS  1:63360  4.498373e+06   
3  Darvin Island                  USGS quads  1:63360  6.287647e+02   
4           None  USGS paper quads, BLM PLSS  1:63360  9.316132e+05   
5           None  USGS paper quads, BLM PLSS  1:63360  1.117682e+07   

    

In [20]:
split_metrics_dir = '/Users/ojlarson/Documents/modis-viirs/split_metrics'
viirs_metrics_dir = '/Users/ojlarson/Documents/modis-viirs/viirs_metrics/v1'

shape_id_column = 'NWRName'

modis_metric_path = str(split_metrics_dir / Path('band_{band}') / Path('{year}_snowyear_metrics_v006.tif_band{band}.tif'))
viirs_metric_path = str(viirs_metrics_dir / Path('{year}') / 'metrics' / Path('{metric_name}_merged_{year}.tif'))
print(modis_metric_path)
viirs_metric_names = ['first_snow_day', 'last_snow_day', 'fss_range', 'longest_css_start', 'longest_css_end', 'longest_css_range', 'snow_days', 'no_snow_days', 'css_segment_num', None, None, 'tot_css_days']
modis_metric_names = ['first_snow_day', 'last_snow_day', 'fss_range', 'longest_css_first_day', 'longest_css_last_day', 'longest_css_day_range', 'snow_days', 'no_snow_days', 'css_segment_num', 'mflag', 'cloud_days', 'tot_css_days']

snow_year = '2015'
band_to_compare = 7

#processor = ZonalProcessor(modis_metric_path, viirs_metric_path, [band_to_compare], [snow_year], viirs_metric_names, modis_metric_names)

#print(processor.get_zonal_stats_df(shp_src, shape_id_column))

/Users/ojlarson/Documents/modis-viirs/split_metrics/band_{band}/{year}_snowyear_metrics_v006.tif_band{band}.tif


In [30]:
combined_stats = get_zonal_stats_df(modis_metric_path, viirs_metric_path, 2015, 7, interior_refuges, shape_id_column, viirs_metric_names)

/Users/ojlarson/Documents/modis-viirs/split_metrics/band_{band}/{year}_snowyear_metrics_v006.tif_band{band}.tif 7 2015 snow_days
Dataset shape: (4391, 7036)
Dataset metadata: {'driver': 'GTiff', 'dtype': 'int16', 'nodata': None, 'width': 7036, 'height': 4391, 'count': 1, 'crs': CRS.from_epsg(3338), 'transform': Affine(500.0, 0.0, -1805447.8214,
       0.0, -500.0, 2570024.1641)}
Dataset metric transform: 
| 500.00, 0.00,-1805447.82|
| 0.00,-500.00, 2570024.16|
| 0.00, 0.00, 1.00|
/Users/ojlarson/Documents/modis-viirs/viirs_metrics/v1/{year}/metrics/{metric_name}_merged_{year}.tif 7 2015 snow_days
Dataset shape: (16977, 22205)
Dataset metadata: {'driver': 'GTiff', 'dtype': 'int16', 'nodata': 0.0, 'width': 22205, 'height': 16977, 'count': 1, 'crs': CRS.from_epsg(3338), 'transform': Affine(375.0, 0.0, -4524375.0,
       0.0, -375.0, 6366000.0)}
Dataset metric transform: 
| 375.00, 0.00,-4524375.00|
| 0.00,-375.00, 6366000.00|
| 0.00, 0.00, 1.00|
assigning sensor and version manually


In [39]:
class MetricDataset:
    def __init__(self, file_template, band, year, sensor, version, metric_names):
        self.file_template=file_template
        self.band=band
        self.year=year
        self.sensor=sensor
        self.version=version
        self.metric_name=metric_names[band - 1]

        self.file_path=Path(file_template.format(band=self.band, year=self.year, metric_name=self.metric_name))

    def load_tiff(self):
        try:
            with rio.open(self.file_path) as src:
                src_transform = src.transform
                src_array = src.read(1)
        except:
            print(f"Cannot find {self.file_path}, probably no metric tiff file...")
            return None
        print("Dataset shape:", src.shape)
        print("Dataset metadata:", src.meta)
        print(f"Dataset metric transform: \n{src_transform}")
        return src_array, src_transform

In [40]:
ds1 = MetricDataset(modis_metric_path, 7, 2015, 'modis', 'new-6', viirs_metric_names)
ds2 = MetricDataset(viirs_metric_path, 7, 2015, 'viirs', 'v1', viirs_metric_names)

In [42]:
ds1.load_tiff()

Dataset shape: (4391, 7036)
Dataset metadata: {'driver': 'GTiff', 'dtype': 'int16', 'nodata': None, 'width': 7036, 'height': 4391, 'count': 1, 'crs': CRS.from_epsg(3338), 'transform': Affine(500.0, 0.0, -1805447.8214,
       0.0, -500.0, 2570024.1641)}
Dataset metric transform: 
| 500.00, 0.00,-1805447.82|
| 0.00,-500.00, 2570024.16|
| 0.00, 0.00, 1.00|


(array([[232, 232, 232, ..., 234, 238, 232],
        [236, 232, 232, ..., 237, 231, 235],
        [232, 235, 239, ..., 237, 231, 231],
        ...,
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]], dtype=int16),
 Affine(500.0, 0.0, -1805447.8214,
        0.0, -500.0, 2570024.1641))

In [37]:
ds1.file_path
ds2.file_path

PosixPath('/Users/ojlarson/Documents/modis-viirs/viirs_metrics/v1/2015/metrics/snow_days_merged_2015.tif')

In [31]:
combined_stats

,min,max,mean,count,non_zero_mean,non_zero_max,non_zero_min,std_dev,non_zero_std_dev,NWRName,sensor,version
0,0.0,250.0,176.774915,73932,185.9396,250,74,42.1931,12.9787,Innoko,modis,new-v6
1,0.0,234.0,189.170123,26475,196.2415,234,75,37.9727,10.3960,Kanuti,modis,new-v6
2,0.0,237.0,185.314847,72845,198.9193,237,55,51.3385,11.0863,Koyukuk,modis,new-v6
3,0.0,232.0,180.316802,33311,185.5357,232,60,33.5141,13.6887,Nowitna,modis,new-v6
4,0.0,347.0,165.263692,15082,193.8638,347,155,71.9069,22.8148,Tetlin,modis,new-v6
5,0.0,260.0,185.884301,180927,203.3687,260,80,57.7376,9.5597,Yukon Flats,modis,new-v6
0,0.0,174.0,106.356188,131425,112.7374,174,1,37.6836,28.0330,Innoko,viirs,v1
1,0.0,188.0,103.792979,47116,109.6285,188,5,36.5163,27.7245,Kanuti,viirs,v1
2,0.0,199.0,108.985881,129474,118.3329,199,1,40.1965,25.4611,Koyukuk,viirs,v1
3,0.0,177.0,96.008209,59203,99.5198,177,1,38.4771,34.4262,Nowitna,viirs,v1
